As I refined my search to limit number of users, this script will allow the updating of the users to download, removing those already downloaded remotely.

On another note, I realised that having so many files is not great, so it may be better on the GH script to merge all the files into pandas frames (TBC)

In [2]:
import pandas as pd
from os.path import join, isfile
from os import listdir
import numpy as np
import pickle
import math

In [3]:
!ls

1. Statistical analysis 2012-16 (Dask).ipynb
1. Statistical analysis 2015 (Dask).ipynb
1. Statistical analysis 2015-Spark.ipynb
2012
2013
2014
2015
2015new
2015statanal.py
2016
3. GH files to download updater.ipynb
Bin_To_Delete
Github Archive Downloader.ipynb
JSON to pandas.ipynb
Unused scripts
build
derby.log
df1216.pkl
ghdatadl.py
ghdatadl_1of4.py
ghdatadl_2of4.py
ghdatadl_3of4.py
ghdatadl_4of4.py
ghuserstodl000000000000.csv
latestusrlist.pkl
metastore_db
numeventsbyuser12-16.pkl
updateduserlist.pkl
user_data
user_data 2
userlist_1of4.pkl
userlist_2of4.pkl
userlist_3of4.pkl
userlist_4of4.pkl
z_old


## Load global list of users to download

In [4]:
global_userlist = np.array(pd.read_csv('1.data/ghuserstodl000000000000.csv')['actor_login'])

In [277]:
len(global_userlist)

1995284

## Load list of files already downloaded

In [5]:
already_dl = np.array([f[:len(f)-5] for f in listdir('1.data/user_data') if \
                       isfile(join('1./data/user_data',f))])

In [6]:
already_dl2 = np.array([f[:len(f)-5] for f in listdir('1.data/user_data2') if \
                        isfile(join('1.data/user_data 2', f))])

In [7]:
all_dls = np.concatenate((already_dl,already_dl2))

In [8]:
len(all_dls) == len(already_dl) + len(already_dl2)

True

In [9]:
len(all_dls)

241551

## Return a matrix of files to download that does not contain already downloaded files

In [283]:
remaining_DLs = np.setdiff1d(global_userlist, all_dls)

In [284]:
len(remaining_DLs)

1791074

In [285]:
already_dl[25]

'0-Eclipse-0'

In [286]:
len(global_userlist)

1995284

In [287]:
len(remaining_DLs)

1791074

In [288]:
print('There are ' + str(len(global_userlist) - len(remaining_DLs)) + ' users still to be downloaded')

There are 204210 users still to be downloaded


In [289]:
pickle.dump(remaining_DLs, open('2.pickles/updateduserlist.pkl', 'wb'))

In [10]:
# Creates a list of already downloaded users within the latest global user list
already_dl = np.intersect1d(global_userlist,all_dls)

In [11]:
len(already_dl)

204210

In [12]:
pickle.dump(already_dl, open('2.pickles/latestusersalreadydownloaded.pkl', 'wb'))

## Calculate time it would take to download remaining users

In [290]:
max_requests = 5000
time_to_dl = 1 # in minutes
delay_between_requests = 60 + time_to_dl # minutes
total_for_each_chunk = time_to_dl + delay_between_requests

In [291]:
chunks = math.ceil(len(remaining_DLs) / max_requests)

In [292]:
total_time = chunks * total_for_each_chunk

In [293]:
time_in_hours = total_time / 60

In [294]:
time_in_hours

370.96666666666664

In [295]:
time_in_days = time_in_hours / 24

In [296]:
time_in_days

15.456944444444444

In [298]:
for i in range(1,6):
    print(str(i)+' instance: '+ str(round(time_in_days/i,1)) + ' days')

1 instance: 15.5 days
2 instance: 7.7 days
3 instance: 5.2 days
4 instance: 3.9 days
5 instance: 3.1 days


## Split remaining downloads into 4 chunks so can download on 4 different instances

In [310]:
remaining_DL_to_split = np.array_split(remaining_DLs, 4)

In [311]:
remaining_DL_1 = remaining_DL_to_split[0]

In [312]:
remaining_DL_2 = remaining_DL_to_split[1]

In [313]:
remaining_DL_3 = remaining_DL_to_split[2]

In [314]:
remaining_DL_4 = remaining_DL_to_split[3]

In [315]:
len(remaining_DLs) == len(remaining_DL_1) + len(remaining_DL_2) + len(remaining_DL_3) + len(remaining_DL_4)

True

In [316]:
len(remaining_DL_1)

447769

In [317]:
len(remaining_DL_2)

447769

In [318]:
len(remaining_DL_3)

447768

In [319]:
len(remaining_DL_4)

447768

In [323]:
pickle.dump(remaining_DL_1, open('2.pickles/userlist_1of4.pkl', 'wb'))

In [324]:
pickle.dump(remaining_DL_2, open('2./pickles/userlist_2of4.pkl', 'wb'))

In [325]:
pickle.dump(remaining_DL_1, open('2./pickles/userlist_3of4.pkl', 'wb'))

In [326]:
pickle.dump(remaining_DL_1, open('2.pickles/userlist_4of4.pkl', 'wb'))

# Sidenote - Get Request Limit for Github to amend data

In [198]:
import requests
import time
from datetime import
from requests.auth import HTTPBasicAuth

In [199]:
GH_username = ''
GH_password = ''

In [264]:
reset = 0

def get_ratelimit():
    global reset
    get_limit = requests.get('https://api.github.com/rate_limit',
                            auth=HTTPBasicAuth(GH_username,
                                              GH_password))
    data = get_limit.json()
    limit = data['rate']['remaining']
    reset = data['rate']['reset']
    return limit, reset

In [265]:
get_ratelimit()

(4982, 1479768316)

In [266]:
timegm(time.gmtime())

1479764739

In [273]:
get_ratelimit()[1] - timegm(time.gmtime())

2033

0

## Return a matrix of relevant files already downloaded